In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
import warnings
#################DATA PREPROCESSING CLASS###################3

from features import FiducialDataProcess, NMF_Data

TA_test_folder_name = input('INSERT NAME OF TA TEST FOLDER; make sure folder is located in /data/ repo')

'''
#This Class is in features.py
class FiducialDataProcess(object):

	def __init__(self, path, num_data, num_features):
		self.path = path # 
		self.num_data = num_data
		self.num_features = num_features
		self.feature_data = np.zeros((num_data, num_features*(num_features-1)))#np.zeros((2500,78*77))

	def euc_dist(self, p1, p2):
		dist = ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)**0.5
		return int(dist)


	def printkeys(self,i):
		data = loadmat(self.path + '{0:04}'.format(i) +  '.mat')
		return data

	def preprocess(self):

		total_features = []
		for i in range(1, self.num_data+1):
			curr_feature = []
			data = loadmat(self.path + '{0:04}'.format(i) +  '.mat')

			if 'faceCoordinatesUnwarped' in data:
				arr = data['faceCoordinatesUnwarped']
			else:
				arr = data['faceCoordinates2']

			for j in range(arr.shape[0]-1):
				for k in range(j+1, arr.shape[0]):
					curr_feature.append(self.euc_dist(arr[j],arr[k]))
			total_features.append(curr_feature)
		return total_features

	def return_features(self):
		t = self.preprocess()
		return t

'''

####################DATA PREPROCESSING SCRIPT ########################## 

path = '../data/' + TA_test_folder_name + '/points/' #SPECIFY FOLDER PATH WHERE THE NEW FIDUCIAL DATAPOINTS ARE
print(path)
check = input('Is this path correct? Please type yes or no')

if check == 'yes':
    print('thank you')
elif check == 'no':
    print('example path should be ../data/TEST_SET/points/')
    print('Please specify EXACT path where the /points/ folder is located in :')
else:
    print('Input error; please rerun entire script again')
    
print('NOTE: SCRIPT WILL NOT RUN IF PATH IS WRONG')    
path_, dirs, files = next(os.walk(path))
file_count = len(files)

num_features = 78
num_data = file_count
feature_array = FiducialDataProcess(path, num_data, num_features)
final_features = np.array(feature_array.return_features())


############################# FEATURE ENGINEERING CLASS###########################################
import pickle
'''
#this class is in features.py
class NMF_Data(object):
	def __init__(self, dat_x, dat_y):
		self.dat_x = dat_x
		self.dat_y = dat_y
		self.nmf_features = []
		self.nmf = None

	def create_nmf(self, reduc_comp=100, test_size=500):
		x_train, x_test, y_train, y_test = train_test_split(self.dat_x, self.dat_y, random_state=1, test_size = test_size)
		print(x_train.shape, x_test.shape)
		
		self.nmf = NMF(n_components=reduc_comp, random_state=0)
		self.nmf.fit(x_train)

		x_train_nmf = self.nmf.transform(x_train)
		x_test_nmf = self.nmf.transform(x_test)

		self.nmf_features.append(x_train_nmf)
		self.nmf_features.append(y_train)
		self.nmf_features.append(x_test_nmf)
		self.nmf_features.append(y_test)

	def nmf_dim_reduc(self, data):
		return self.nmf.transform(data)

	def get_nmf_features(self):
		return self.nmf_features

	def save_nmf(self, filename):
		self.create_nmf()
		np.save(filename, self.nmf_features)
'''
#################################### FEATURE ENGINEERING SCRIPT ##########################################

print('PREPROCESS DATA STARTING')
url1 = 'dense_data_type_and_emot.csv'
original_data = np.genfromtxt(url1, delimiter=',')
np.random.seed(0)
np.random.shuffle(original_data)	
features_init = original_data[:,2:]
features = features_init / features_init.max(axis=0)
labels = original_data[:,0:2] 

orig_bin_nmf_features = NMF_Data(features, labels)
orig_bin_nmf_features.create_nmf(reduc_comp=100)
new_bin_nmf_features = orig_bin_nmf_features.nmf_dim_reduc(final_features)
x_test_set_bin = new_bin_nmf_features

orig_emot_nmf_features = NMF_Data(features, labels)
orig_emot_nmf_features.create_nmf(reduc_comp=300)
new_emot_nmf_features = orig_emot_nmf_features.nmf_dim_reduc(final_features)
x_test_set_emot = new_emot_nmf_features

test_set_len = len(x_test_set_bin)

print('PREPROCESS DATA HAS FINISHED')

In [2]:
data = pd.read_csv('dense_data_type_and_emot.csv', header = None)
y_label = np.array(data[[0,1]])
data.drop(data.columns[[0,1]],axis=1,inplace=True)

In [3]:
feature = []
for item in list(data.columns):
    name = 'feature' + str(int(item)-1)
    feature.append(name)

data.columns = feature

In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = {'n_estimators':[50,100,200],
                 'max_depth':[5,10,15],
                 'min_samples_split':np.arange(2,14,4),
                 'max_features':np.arange(40,100,20),
                 }

rf_clf = RandomForestClassifier(**rf_param_grid)
rf_grid = GridSearchCV(rf_clf,
                    param_grid=rf_param_grid,
                    cv=3,
                    verbose=1,
                    scoring='accuracy')

rf_grid.fit(data, y_label[:,0])
rf_grid.best_params_, rf_grid.best_score_

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=1)]: Done 243 out of 243 | elapsed: 11.6min finished


({'max_depth': 10,
  'max_features': 40,
  'min_samples_split': 2,
  'n_estimators': 50},
 0.65)

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

gbdt_param_grid = {'learning_rate':[0.01,0.1,0.5,1],
                   'n_estimators':[50,100,200],
                   'max_depth':[5,10,15],
                   'max_features':np.arange(20,100,20),
                   }

gbdt_clf = GradientBoostingClassifier(**gbdt_param_grid)

gbdt_grid = GridSearchCV(gbdt_clf,
                    param_grid=gbdt_param_grid,
                    cv=3,
                    verbose=1,
                    scoring='accuracy')

gbdt_grid.fit(data, y_label[:,0])
gbdt_grid.best_params_, gbdt_grid.best_score_

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=1)]: Done 432 out of 432 | elapsed: 20.0min finished


({'learning_rate': 0.01,
  'max_depth': 5,
  'max_features': 20,
  'n_estimators': 50},
 0.6768)

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.grid_search import GridSearchCV
import xgboost as xgb
warnings.filterwarnings('ignore')

xgb_param_grid = {
                  'max_depth':[5,10,15],
                  #'colsample_bytree': np.linspace(0.1, 0.9, 9),
                  #'subsample': np.linspace(0.1, 0.9, 9),
                  'learning_rate' : [0.01,0.1,0.5,1],
                  'n_estimators':[20,50,100,200],
                  'max_features':np.arange(10,70,20),
                  }

xgb_clf = xgb.XGBClassifier(**xgb_param_grid)

xgb_grid = GridSearchCV(xgb_clf,
                    param_grid=xgb_param_grid,
                    cv=3,
                    verbose=1,
                    scoring='accuracy')

xgb_grid.fit(data, y_label[:,0])
xgb_grid.best_params_, xgb_grid.best_score_

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=1)]: Done 432 out of 432 | elapsed: 272.9min finished


({'learning_rate': 0.5,
  'max_depth': 5,
  'max_features': 10,
  'n_estimators': 200},
 0.6496)

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = {'n_estimators':[50,100,200],
                 'max_depth':[5,10,15],
                 'min_samples_split':np.arange(2,14,4),
                 'max_features':np.arange(40,100,20),
                 }

rf_clf = RandomForestClassifier(**rf_param_grid)
rf_grid = GridSearchCV(rf_clf,
                    param_grid=rf_param_grid,
                    cv=3,
                    verbose=1,
                    scoring='accuracy')

rf_grid.fit(data, y_label[:,1])
rf_grid.best_params_, rf_grid.best_score_

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=1)]: Done 243 out of 243 | elapsed: 14.2min finished


({'max_depth': 15,
  'max_features': 60,
  'min_samples_split': 2,
  'n_estimators': 200},
 0.3768)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gbdt_param_grid = {'learning_rate':[0.01,0.1,0.5,1],
                   'n_estimators':[50,100,200],
                   'max_depth':[5,10,15],
                   'max_features':np.arange(20,100,20),
                   }

gbdt_clf = GradientBoostingClassifier(**gbdt_param_grid)

gbdt_grid = GridSearchCV(gbdt_clf,
                    param_grid=gbdt_param_grid,
                    cv=3,
                    verbose=1,
                    scoring='accuracy')

gbdt_grid.fit(data, y_label[:,1])
gbdt_grid.best_params_, gbdt_grid.best_score_

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=1)]: Done 432 out of 432 | elapsed: 171.1min finished


({'learning_rate': 0.1,
  'max_depth': 5,
  'max_features': 80,
  'n_estimators': 200},
 0.3688)